In [2]:
%pip install imutils opencv-python numpy face_recognition 
import dlib
dlib.DLIB_USE_CUDA = True
print(f"CUDA disponível: {dlib.DLIB_USE_CUDA}")
#print(f"Número de GPUs disponíveis: {dlib.cuda.get_num_devices()}")

Note: you may need to restart the kernel to use updated packages.
CUDA disponível: True


In [3]:
import os
import cv2
import face_recognition
import pickle
import numpy as np
from imutils import paths

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-5prnabg4/dlib_622639a74c8e49a8a6f0f43c1946fb7e/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
def train_model(training_dir="Images"):
    known_encodings = []
    known_names = []
    
    # Iterar pelas pastas de cada pessoa em Images/
    for person_name in os.listdir(training_dir):
        person_path = os.path.join(training_dir, person_name)
        
        if not os.path.isdir(person_path):
            continue
        
        # Iterar pelas imagens de cada pessoa
        for image_path in paths.list_images(person_path):
            image = cv2.imread(image_path)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Detectar rostos na imagem e obter os encodings
            boxes = face_recognition.face_locations(rgb_image, model='hog')
            encodings = face_recognition.face_encodings(rgb_image, boxes)
            
            for encoding in encodings:
                known_encodings.append(encoding)
                known_names.append(person_name)
    
    # Salvar os encodings treinados em um arquivo
    data = {"encodings": known_encodings, "names": known_names}
    with open("face_encodings.pkl", "wb") as f:
        pickle.dump(data, f)
    
    print("[INFO] Treinamento concluído e encodings salvos.")

In [ ]:
def recognize_faces(test_dir="Test_Images", encodings_file="face_encodings.pkl"):
    # Carregar os encodings treinados
    with open(encodings_file, "rb") as f:
        data = pickle.load(f)
    
    for image_path in paths.list_images(test_dir):
        image = cv2.imread(image_path)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detectar rostos na imagem de teste
        boxes = face_recognition.face_locations(rgb_image, model='hog')
        encodings = face_recognition.face_encodings(rgb_image, boxes)
        
        # Comparar com os encodings conhecidos
        names = []
        for encoding in encodings:
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            name = "Nao reconhecido"
            
            if True in matches:
                matched_idxs = [i for i, match in enumerate(matches) if match]
                counts = {}
                
                for i in matched_idxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1
                
                name = max(counts, key=counts.get)
            
            names.append(name)
        
        # Exibir resultados
        for ((top, right, bottom, left), name) in zip(boxes, names):
            cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 5)
            cv2.putText(image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 5)
        
        cv2.imshow("Image", image)
        cv2.waitKey(0)
    
    cv2.destroyAllWindows()

In [ ]:
train_model("Images")

In [ ]:
recognize_faces("Test_Images")